In [1]:
# default_exp python_decorators

# Python decorators introduction

> Some examples on how to use decorator in Python.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, discrete choice, statsmodels]

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import functools
import logging
import random
import time
from collections import defaultdict

### Functions are first class objects

In [4]:
#export
# Function as a parameter
def hello(name, logger):
    logger(f"Hello {name}")
    
hello("world", logger=print)

Hello world


In [5]:
hello("First log",logger=logging.warning)

In [6]:
hello("Second log",logger=logging.info)

In [7]:
with open("hello.txt", mode="w") as file:
    hello("Files", logger=file.write)
!cat hello.txt

Hello Files

In [8]:
def reversed_print(text):
    print(text[::-1].capitalize())

reversed_print("Hi there")

Ereht ih


In [9]:
hello("Danh", logger=reversed_print)

Hnad olleh


In [10]:
hello("world", logger=print)

Hello world


### Inner functions

In [11]:

def outer():
    print("Hi from the outer")
    y = 2020
    def inner():
        print("Hello from the inner")
        print(f"This year is {y}")
    inner()
    return inner

In [12]:
outer()

Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [13]:
inside = outer()
inside

Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [14]:
inside()

Hello from the inner
This year is 2020


#### Manipulate functions

In [15]:
def hi(func):
    print(f"Hello {func.__name__}")
hi(outer)

Hello outer


In [16]:
inside

<function __main__.outer.<locals>.inner()>

In [17]:
hi(inside)

Hello inner


In [18]:
def hi(func):
    print(f"Hello {func.__name__}")
    return func

In [19]:
hi(outer)()

Hello outer
Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [20]:
new_outer = hi(outer)

Hello outer


In [21]:
new_outer is outer

True

In [22]:
def wrapper(func):
    def _wrapper():
        print(f"Before {func.__name__}")
        func()
        print(f"After {func.__name__}")
    return _wrapper

wrapper(outer)

<function __main__.wrapper.<locals>._wrapper()>

In [23]:
outer()

Hi from the outer
Hello from the inner
This year is 2020


<function __main__.outer.<locals>.inner()>

In [24]:
new_outer = wrapper(outer)


In [25]:
new_outer()

Before outer
Hi from the outer
Hello from the inner
This year is 2020
After outer


In [26]:
outer = wrapper(outer)

In [27]:
outer

<function __main__.wrapper.<locals>._wrapper()>

In [28]:
outer()

Before outer
Hi from the outer
Hello from the inner
This year is 2020
After outer


#### Sugar syntax

In [29]:
# Same as: outer2 = wrapper(outer2)
@wrapper
def outer2():
    print("Hi from the outer")
    y = 2020
    def inner():
        print("Hello from the inner")
        print(f"This year is {y}")
    inner()
    return inner

In [30]:
outer2()

Before outer2
Hi from the outer
Hello from the inner
This year is 2020
After outer2


In [31]:
@wrapper
def dice_roll():
    return random.randint(1,6)

In [32]:
dice_roll()

Before dice_roll
After dice_roll


In [33]:
dice_roll.__name__

'_wrapper'

In [34]:
def wrapper(func):
    def _wrapper(*args, **kwargs):
        print(f"Before {func.__name__}")
        func(*args, **kwargs)
        print(f"After {func.__name__}")
    return _wrapper

@wrapper
def hello(name):
    print(f"Hello {name}")

hello("world")

Before hello
Hello world
After hello


In [35]:
def wrapper(func):
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        print(f"Before {func.__name__}")
        value = func(*args, **kwargs)
        print(f"After {func.__name__}")
        return value
#    _wrapper.__name__ = func.__name__
    return _wrapper

@wrapper
def dice_roll():
    """ Roll a 6-sided dice"""
    return random.randint(1,6)

dice_roll()

Before dice_roll
After dice_roll


3

In [36]:
dice_roll.__name__

'dice_roll'

### Task 1: Time counter

In [37]:
#export
def wrapper(func):    
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        # Before func
        value = func(*args, **kwargs)
        # After func
        return value
    return _wrapper

In [38]:
#export
def timer(func):
    @functools.wraps(func)
    def _timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        print(f"Elapsed time: {toc-tic:.2f} seconds")
        return value
    return _timer


@timer
def waste_time(number):
    total = 0
    for num in range(number):
        total += sum(n for n in range(num))
    return total

In [39]:
# Test
waste_time(300)

Elapsed time: 0.00 seconds


4455100

In [40]:
waste_time(3000)

Elapsed time: 0.15 seconds


4495501000

### Task 2: trace

In [41]:
#export

def get_params(*args, **kwargs):
    ars = [repr(a) for a in args]
    kws = [f"{k}={repr(v)}" for k,v in kwargs.items()]
    return ', '.join(ars + kws)

def trace(func):
    """Show the trace of function calls"""
    name = func.__name__
    @functools.wraps(func)
    def _trace(*args, **kwargs):
        print(f"Calling {name}({get_params(*args,**kwargs)})")
        value = func(*args, **kwargs)
        print(f"{name} returned {value}")
        return value
    return _trace

GREETINGS = ["ABC", "EHLLO", "NO!!!"]

@trace
def greet(name, greeting="Hello"):
    return f"{greeting} {name}"


@trace
@timer
def random_greet(name="Emily"):
    greeting = random.choice(GREETINGS)
    return greet(name, greeting=greeting)

@trace
def greet_many(number):
    return [random_greet() for _ in range(number)]


In [42]:
greet("world")

Calling greet('world')
greet returned Hello world


'Hello world'

In [43]:
greet(name="world", greeting="def")

Calling greet(name='world', greeting='def')
greet returned def world


'def world'

In [44]:
random_greet()

Calling random_greet()
Calling greet('Emily', greeting='EHLLO')
greet returned EHLLO Emily
Elapsed time: 0.00 seconds
random_greet returned EHLLO Emily


'EHLLO Emily'

In [45]:
greet_many(3)

Calling greet_many(3)
Calling random_greet()
Calling greet('Emily', greeting='EHLLO')
greet returned EHLLO Emily
Elapsed time: 0.00 seconds
random_greet returned EHLLO Emily
Calling random_greet()
Calling greet('Emily', greeting='ABC')
greet returned ABC Emily
Elapsed time: 0.00 seconds
random_greet returned ABC Emily
Calling random_greet()
Calling greet('Emily', greeting='ABC')
greet returned ABC Emily
Elapsed time: 0.00 seconds
random_greet returned ABC Emily
greet_many returned ['EHLLO Emily', 'ABC Emily', 'ABC Emily']


['EHLLO Emily', 'ABC Emily', 'ABC Emily']

In [46]:
random_greet.__name__

'random_greet'

### TASK 3: Register

In [47]:
#export
REGISTERED = {}

def register(func):
    name = func.__name__
    if name not in REGISTERED: REGISTERED[name] = func
    return func

@register
def true_or_false(text):
    tf_values = {
        True: {"true", "on", "yes", "1"},
        False: {"false", "off", "no", "0"}
    }
    for tf, values in tf_values.items():
        if text.lower() in values:
            return tf

@register
def reversed(text):
    return text[::-1].capitalize()

@register
def robber_language(text):
    consonants = "bcdfghlmnpqrstvwxyz"
    return "".join(
        f"{c}o{c.lower()}" if c.lower() in consonants else c
        for c in text
    )

# text = input("Please input a text:")

# while True:
#     print(f"Parsers: {', '.join(REGISTERED)}")
#     parser = input("Choose a parser: ")
#     if parser in REGISTERED: break


# parser_func = REGISTERED[parser]
# print(parser_func(text))
# parser, text, REGISTERED[parser]

In [48]:
REGISTERED['robber_language']("decorator")

'dodecocororatotoror'

In [49]:
words = ['map', 'mid', 'acb', 'gqre', 'hello', 'anh', 'minh']
ls = {}

for w in words:
    if w[0] not in ls.keys(): ls[w[0]] = [w]
    else: ls[w[0]].append(w)
ls

{'m': ['map', 'mid', 'minh'],
 'a': ['acb', 'anh'],
 'g': ['gqre'],
 'h': ['hello']}

In [50]:
ls2 = {}
for w in words:
    ls2.setdefault(w[0],[]).append(w)

In [51]:
def test_eq(x,y):
        assert x == y

In [52]:
test_eq(ls2, ls)

In [53]:
def get_second(w): return w[1] if len(w) > 1 else w


In [54]:
ls3 = {}
for w in words:
    ls3.setdefault(get_second(w),[]).append(w)

In [55]:
a = '1'
hash(a)

2508636743907904454

In [56]:
hash(tuple(set(dir(dict))))

4942437628418865655

In [57]:

def get_dict1(words):
    ls2 = {}
    for w in words:
        ls2.setdefault(w[0],[]).append(w)
    return ls2

def get_dict2(words):
    ls4 = defaultdict(list)
    for w in words:
        ls4[w[0]].append(w)
    return ls4

test_eq(get_dict1(words),get_dict2(words))

In [58]:
%timeit get_dict1(words)

858 ns ± 6.07 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [59]:
%timeit get_dict2(words)

1.07 µs ± 12 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [60]:
count = defaultdict(lambda : 4)
type(count)

collections.defaultdict

In [61]:
count[2] = 23

In [62]:
count[3], count

(4, defaultdict(<function __main__.<lambda>()>, {2: 23, 3: 4}))

### Reference: 
Geir Arne Hjelle - Introduction to Decorators: Power Up Your Python Code. Link: https://www.youtube.com/watch?v=T8CQwGIsrx4